In [ ]:
okru_soup = soup_from_path("/Users/timdunn/Downloads/okru comedy.html")

In [107]:
all_links = okru_soup.find_all("a", {"class": "video-card_lk"})

In [108]:
len(all_links)

967

In [102]:
from __future__ import unicode_literals
import youtube_dl

In [103]:
def download_videos(video_links, ydl_opts={}):
    for video_link in video_links:
        download_a_video(video_link, ydl_opts)

In [104]:
def download_a_video(video_link, ydl_opts={}):
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_link])

In [105]:
def okru_links(okru_soup):
    return [link_tag["href"] for link_tag in okru_soup.find_all("a", {"class": "video-card_lk"})]

In [73]:
download_a_video("https://ok.ru/video/4088176511578")

[Odnoklassniki] 4088176511578: Downloading webpage
[Odnoklassniki] 4088176511578: Downloading m3u8 information
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 616
[download] Destination: Pin... 1988, Sandor Stern VOSE-4088176511578.mp4
[download] 100% of 923.83MiB in 25:304.32KiB/s ETA 00:00204


In [ ]:
okru_links(okru_soup)

In [ ]:
links = okru_links(okru_soup)

In [99]:
links.index('https://ok.ru/video/3693927074342')

162

In [ ]:
download_videos(links, {"retries": 5})

In [39]:
from __future__ import unicode_literals
import youtube_dl

ydl_opts = {}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(["https://ok.ru/video/35298085598"])

[Odnoklassniki] 35298085598: Downloading webpage
[Odnoklassniki] 35298085598: Downloading m3u8 information
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 572
[download] Destination: The.House.Of.The.Devil.2009.LiMiTED.720p.BRRip.800MB.MkvCage.com-35298085598.mp4
[download] 100% of 962.14MiB in 27:198.34KiB/s ETA 00:005015


In [40]:
ydl_opts = {}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(["https://ok.ru/video/c2905245"])

[generic] c2905245: Requesting header


[generic] c2905245: Downloading webpage
[generic] c2905245: Extracting information


ERROR: Unsupported URL: https://ok.ru/video/c2905245


DownloadError: ERROR: Unsupported URL: https://ok.ru/video/c2905245

In [5]:
!youtube-dl --get-subs https://www.youtube.com/watch?v=q6-hxlvuWZc

zsh:1: no matches found: https://www.youtube.com/watch?v=q6-hxlvuWZc


In [3]:
import pandas as pd
import numpy as np
import glob
import os
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
def flatten(some_list):
    return [item for sublist in some_list for item in sublist]

In [32]:
def df_from_path(path):
    soup = soup_from_path(path)
    df = tubi_df(soup)
    df = df.dropna()
    return df.drop_duplicates()

In [31]:
def soup_from_path(path):
    with open(path) as fp:
        soup = BeautifulSoup(fp, 'html.parser')
    return soup

In [30]:
def a_tags_movies(soup):
    return soup.find_all('a', {"class": "web-content-tile__title"})

In [29]:
def url_title_a_tags(a_tags):
    title_urls = [{"url": a_tag['href'], "movie_title": a_tag.text } for a_tag in a_tags]
    return pd.DataFrame(data=title_urls)

In [28]:
def info_divs(soup):
    return soup.find_all('div', {"class": "web-content-tile__content-digest"})

In [27]:
def tubi_df(soup):
    data_dicts = [info_from_info_div(info_div) for info_div in info_divs(soup)]
    return pd.DataFrame(data=data_dicts)

In [26]:
def year_from_info_div(info_div):
    return info_div.find("div", {"class": "web-content-tile__year"}).text

In [25]:
def info_from_info_div(info_div):
    try:
        link_tag = info_div.find("a")
        return {
            "url": link_tag["href"],
            "movie_title": link_tag.text,
            "movie_year": year_from_info_div(info_div)
        }
    except:
        return {
            "url": pd.NA,
            "movie_title": pd.NA,
            "movie_year": pd.NA
        }

In [24]:
def get_full_df_from_paths(paths):
    dfs = [df_from_path(path) for path in paths]
    for path in paths[1:]:
        df = pd.concat(dfs).drop_duplicates().reset_index(drop=True)
    return df

In [23]:
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

In [22]:
def get_files_from_paths(paths):
    return flatten([listdir_nohidden(path) for path in paths])

In [21]:
def unique_tubi_urls(df, paths_to_search):
    return df.loc[df["url"].isin(get_files_from_paths(paths_to_search))]

In [20]:
def movie_has_a_path(movie_title, path):
    return movie_title.split("(")[0] in path

In [19]:
def movie_has_any_path(movie_title, paths):
    try:
        return any([movie_has_a_path(movie_title, path) for path in paths])
    except:
        return False

In [16]:
def divs_for_archives(archives_soup):
    return archives_soup.find_all("div", {"class": "item-ttl"})

In [17]:
def link_tags_archives(archives_soup):
    return [div.find("a") for div in divs_for_archives(archives_soup)]

In [18]:
def dict_for_archives(a_tag):
    try:
        return {"url": a_tag["href"], "movie_title": a_tag["title"]}
    except:
        return {"url": np.nan, "movie_title": np.nan}

In [12]:
def df_for_archives(archives_soup):
    data_dicts = [dict_for_archives(a_tag) for a_tag in link_tags_archives(archives_soup)]
    df = pd.DataFrame(data=data_dicts)
    df = df.dropna()
    return df.drop_duplicates()

In [11]:
def get_page_url(url):
    return requests.get(url)

In [10]:
def get_soup_url(url):
    return BeautifulSoup(get_page_url(url).content, "html.parser")

In [13]:
def get_download_tag(url):
    try:
        soup = get_soup_url(url)
        return soup.find("a", {"class": "format-summary download-pill", "href": re.compile("\.mp4")})
    except:
        return None

In [14]:
def archive_download_link(url):
    download_tag = get_download_tag(url)
#     print("after we get download tag")
    if download_tag is not None:
#         print("inside download tag thing")
#         print(download_tag)
#         print(download_tag["href"])
        return "https://archive.org{}".format(download_tag["href"])
    else:
        return None

In [15]:
def download_video(file_name, url):
    try:
#         print("url is {}".format(url))
        r = requests.get(url, stream = True)
        #download started
#         print("entered after requests")
        with open(file_name, 'wb') as f:
            for chunk in r.iter_content(chunk_size = 1024*1024):
                if chunk:
                    f.write(chunk)
        print ("%s downloaded!\n"%file_name)
    except:
        print("An Error Downloading %s"%file_name)

In [ ]:
def download_videos_from_df(df, path):
    try:
        for index, row in df.iterrows():
            movie_path = "{}/{}.mp4".format(path, row["movie_title"])
            download_video(movie_path, archive_download_link(row["url"]))
        print("All videos Downloaded successfully")
    except:
        print("There was an error downloaded all the movies")

In [ ]:
path = "/Users/timdunn/Downloads/Feature Films _ Free Movies _ Free Download, Borrow and Streaming _ Internet Archive.html"
archives_soup = soup_from_path(path)

In [ ]:
archives_df = df_for_archives(archives_soup)

In [ ]:
archives_df.loc[archives_df["movie_title"] == "Desperate Cargo"].index[0]

In [ ]:
download_videos_from_df(archives_df.loc[2162:4000], "/Users/timdunn/Desktop/archive movie downloads")

In [ ]:
tubi_paths = [
    "/Users/timdunn/Downloads/tubi anime.html",
    "/Users/timdunn/Downloads/tubi award.html",
    "/Users/timdunn/Downloads/tubi bollywood.html",
    "/Users/timdunn/Downloads/tubi classics.html",
    "/Users/timdunn/Downloads/tubi comedy.html",
    "/Users/timdunn/Downloads/tubi crime.html",
    "/Users/timdunn/Downloads/tubi cult.html",
    "/Users/timdunn/Downloads/tubi drama.html",
    "/Users/timdunn/Downloads/tubi faith.html",
    "/Users/timdunn/Downloads/tubi family.html",
    "/Users/timdunn/Downloads/tubi featured.html",
    "/Users/timdunn/Downloads/tubi foreign.html",
    "/Users/timdunn/Downloads/tubi holiday.html",
    "/Users/timdunn/Downloads/tubi indie.html",
    "/Users/timdunn/Downloads/tubi kids.html",
    "/Users/timdunn/Downloads/tubi lgbtq.html",
    "/Users/timdunn/Downloads/tubi lifestyle.html",
    "/Users/timdunn/Downloads/tubi martial arts.html",
    "/Users/timdunn/Downloads/tubi music.html",
    "/Users/timdunn/Downloads/tubi other spanish.html",
    "/Users/timdunn/Downloads/tubi popular.html",
    "/Users/timdunn/Downloads/tubi preschool.html",
    "/Users/timdunn/Downloads/tubi reality tv.html",
    "/Users/timdunn/Downloads/tubi sci fi.html",
    "/Users/timdunn/Downloads/tubi spanish family.html",
    "/Users/timdunn/Downloads/tubi sports.html",
    "/Users/timdunn/Downloads/tubi thriller.html",
    "/Users/timdunn/Downloads/tubi weekly.html",
    "/Users/timdunn/Downloads/tubi westerns.html"
]

In [ ]:
divs[0].find("div", {"class": "web-content-tile__year"}).text

In [ ]:
searching_paths = [
    "/Volumes/Transcend/",
    "/Volumes/Transcend/2010s - 2020s movies/",
    "/Volumes/Transcend/2010s - 2020s new movies/",
    "/Volumes/Backup Plus/1940s-50s-60s movies/",
    "/Volumes/Backup Plus/1940s-50s-60s new movies/",
    "/Volumes/Elements/in between foreign films/",
    "/Volumes/Elements/1970s - 2000s movies /",
    "/Volumes/Elements/1970s - 2000s new movies /"
]